In [6]:
import requests
import pandas as pd

In [7]:
data_url='https://wapi.gogoro.com/tw/api/vm/list'
r = requests.get(data_url,stream = True) #stream=True #開啟時時續續的下載的方式
r.encoding = 'utf-8'
print(r.encoding)
with open('./data.txt','wb') as f: 
    for chunk in r.iter_content(chunk_size=32): #chunk_size #設定每次下載檔案的大小
        f.write(chunk)  #每一次迴圈儲存一次下載下來的內容
f.close

utf-8


<function BufferedWriter.close>

In [1]:
# import os
# import sys
# import fileinput
# f1 = open('./data.txt', 'r')
# f2 = open('./new_data.txt', 'w')
# for line in f1:
#     f2.write(line.replace("\\", 'new_text'))
# f1.close()
# f2.close()

In [14]:
import json
with open('./new_data.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)
print(df.info(verbose=True))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2255 entries, 0 to 2254
Data columns (total 4 columns):
result        2255 non-null int64
message       2255 non-null object
data          2255 non-null object
error_code    0 non-null object
dtypes: int64(1), object(3)
memory usage: 70.6+ KB
None
